In [1]:
!pip install dfply
import pandas as pd
import numpy as np
import math
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statistics 
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import pearsonr
import plotly.graph_objects as go
import warnings
from scipy.stats import shapiro
from datetime import date, timedelta

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
sofifa_head = pd.read_pickle('/content/drive/My Drive/Capstone/Data/Clean/sofifa_head_Clean.pkl')

In [0]:
players = pd.read_pickle('/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl')

In [0]:
markval = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_detailed.pkl")

In [0]:
players = players[players.sofifa_id.notnull()]
players.sofifa_id = players.sofifa_id.astype("int") 

In [0]:
sofifa_head["year"] = sofifa_head["game_year"].apply(lambda x : x.year) 

In [0]:
sofifa_head = sofifa_head[["sofifa_id","short_name","overall", "potential",
                           "weak_foot","skill_moves","work_rate","year"]]

In [0]:
sofifa_head = sofifa_head.drop_duplicates()

In [0]:
sofifa_head = pd.merge(sofifa_head,players[["tm_id","sofifa_id"]], on = "sofifa_id")

In [0]:
sofifa_head = pd.merge(sofifa_head,markval, on = ["tm_id","year"])

In [0]:
def scatter(df, xvar, yvar, labels, title, facet_var = "main_field_position",
            facet = False):
  transfers_plot = ( df >> mask(X[xvar].notnull(), X[yvar].notnull()))
  if(facet):
    fig = px.scatter(transfers_plot, xvar, yvar,hover_data = ["short_name","age","nationality","year",xvar,yvar],
                    labels = {xvar:"", yvar:""},
                    title = title, color = facet_var ,
                    facet_col = facet_var, facet_col_wrap = 2)
  else: 
    fig = px.scatter(transfers_plot, xvar, yvar,hover_data = ["short_name","age","nationality","year",xvar,yvar],
                  labels = labels,
                  title = title, color = "main_field_position" )
    
  fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
  fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
  corr, _ = pearsonr(transfers_plot[xvar],transfers_plot[yvar])
  print("Correlation between the variables",corr)
  fig.show()
  return fig

# Fifa rating and market value correlation

In [62]:
rating_mv = scatter(sofifa_head, "overall","mv", {"overall":"Player's overall rating in Fifa",
                                      "mv":"Player's market value"},"Player's market value vs Fifa's overall rating")

Output hidden; open in https://colab.research.google.com to view.

As we can see player's fifa rating is mostly positively correlated to his market value

In [37]:
rating_mv = scatter(sofifa_head, "overall","mv", {"overall":"Player's overall rating in Fifa",
                                      "mv":"Player's market value"},"Player's market value vs fifa overall rating"
                                      ,facet = True)

Output hidden; open in https://colab.research.google.com to view.

The correlation is most explicitly visible among attackers and midfielders

In [40]:
rating_mv = scatter(sofifa_head, "overall","mv", {"overall":"Player's overall rating in Fifa",
                                      "mv":"Player's market value"},"Player's market value vs fifa overall rating"
                                      ,facet = True, facet_var = "continent")

Output hidden; open in https://colab.research.google.com to view.

If we take a look at the correlation by each continent, we can again see Europe and South America dominationg, wtih African player's also getting closer to South America's situation.

In [0]:
sofifa_corr = sofifa_head.groupby("year")[["overall","mv"]].corr().reset_index()
sofifa_corr.drop(["level_1","overall"], axis = 1, inplace = True)
sofifa_corr = sofifa_corr[sofifa_corr.mv != 1]
sofifa_corr.columns = ["year","corr"]

In [51]:
fig = px.line(sofifa_corr, "year", "corr",
              labels = {"corr":"Correlation score",
                        "year":""},
              title = "Correlation between player's fifa rating and market value over time")
fig.show()

As we can see the correlation between the player's rating and his market value is mostly decreasing over years, and in this 5 year region there was only one year when the correlation between the player's rating in fifa and his market value increased over the previous year. The year was 2018,but after this year the correlation started to decrease dramatically each year reaching lower values.